In [ ]:
import numpy as np
import pandas as pd
import time 
import random
from sklearn.model_selection import train_test_split
from sklearn import datasets, preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
ds=pd.read_csv('/content/drive/MyDrive/diabetes.csv')
ds.head(15)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


# Import Data

In [ ]:

x = ds.drop(['Outcome'], axis = 1).values
y = ds['Outcome'].values
y = y.reshape(y.shape[0], 1)
#Spliting Data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.25,random_state=0)
#Standardize data
from sklearn.preprocessing import StandardScaler 
scale = StandardScaler()
x_train_std = scale.fit_transform(x_train)
x_test_std = scale.transform(x_test)


In [ ]:
print(y_train[0][0])

0


In [ ]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

# Feed Forward

In [ ]:
def FF(HL,OL,W,B,x):
  INL=len(x)
  hidden_layer=np.zeros((HL))
  output_layer=np.zeros((OL))
  witr=0
  bitr=0
  for i in range(HL):
    sum=0
    for j in range(INL):
      sum=sum+(x[j]*W[witr])
      witr=witr+1
    sum=sum.sum()
    hidden_layer[i]=sigmoid(sum+B[bitr])
    bitr=bitr+1

  for i in range(OL):
    output=0
    for j in range(HL):
      output=output+(hidden_layer[j]*W[witr])
      witr = witr + 1
    output=output.sum()
    output_layer[i]=sigmoid(output+B[bitr])
    bitr=bitr+1
  return output_layer

In [ ]:
NIL=8
HL=18
OL=1
#144+18+19=181

In [ ]:
def MLP(solution):
      fitness=0
      w=(NIL*HL)+(HL*OL)
      b=(NIL*HL)+HL+(HL*OL)+OL
      W=solution[0:w]
      B=solution[w:b]
      for idx,x in enumerate(x_train_std):
        actualValue=FF(HL,OL,W,B,x)

        fitness+=pow((actualValue[0]-y_train[idx][0]),2)

      fitness=fitness/len(x_train)
      
      return fitness

# GWO Algorithm

In [ ]:
import random
import numpy as np
import math
import time

class solution:
    def __init__(self):
        self.best = 0
        self.bestIndividual = []
        self.convergence = []
        self.optimizer = ""
        self.objfname = ""
        self.startTime = 0
        self.endTime = 0
        self.executionTime = 0
        self.lb = 0
        self.ub = 0
        self.dim = 0
        self.popnum = 0
        self.maxiers = 0
def GWO(objf, lb, ub, dim, SearchAgents_no, Max_iter):

    # Max_iter=1000
    # lb=-100
    # ub=100
    # dim=30
    # SearchAgents_no=5

    # initialize alpha, beta, and delta_pos
    Alpha_pos = np.zeros(dim)
    Alpha_score = float("inf")

    Beta_pos = np.zeros(dim)
    Beta_score = float("inf")

    Delta_pos = np.zeros(dim)
    Delta_score = float("inf")

    if not isinstance(lb, list):
        lb = [lb] * dim
    if not isinstance(ub, list):
        ub = [ub] * dim

    # Initialize the positions of search agents
    Positions = np.zeros((SearchAgents_no, dim))
    for i in range(dim):
        Positions[:, i] = (
            np.random.uniform(0, 1, SearchAgents_no) * (ub[i] - lb[i]) + lb[i]
        )

    Convergence_curve = np.zeros(Max_iter)
    s = solution()

    # Loop counter
    print('GWO is optimizing  "' + objf.__name__ + '"')

    timerStart = time.time()
    s.startTime = time.strftime("%Y-%m-%d-%H-%M-%S")
    # Main loop
    for l in range(0, Max_iter):
        for i in range(0, SearchAgents_no):

            # Return back the search agents that go beyond the boundaries of the search space
            for j in range(dim):
                Positions[i, j] = np.clip(Positions[i, j], lb[j], ub[j])

            # Calculate objective function for each search agent
            fitness = objf(Positions[i, :])

            # Update Alpha, Beta, and Delta
            if fitness < Alpha_score:
                Delta_score = Beta_score  # Update delte
                Delta_pos = Beta_pos.copy()
                Beta_score = Alpha_score  # Update beta
                Beta_pos = Alpha_pos.copy()
                Alpha_score = fitness
                # Update alpha
                Alpha_pos = Positions[i, :].copy()
                

            if fitness > Alpha_score and fitness < Beta_score:
                Delta_score = Beta_score  # Update delte
                Delta_pos = Beta_pos.copy()
                Beta_score = fitness  # Update beta
                Beta_pos = Positions[i, :].copy()

            if fitness > Alpha_score and fitness > Beta_score and fitness < Delta_score:
                Delta_score = fitness  # Update delta
                Delta_pos = Positions[i, :].copy()

        a = 2 - l * ((2) / Max_iter)
        # a decreases linearly fron 2 to 0

        # Update the Position of search agents including omegas
        for i in range(0, SearchAgents_no):
            for j in range(0, dim):

                r1 = random.random()  # r1 is a random number in [0,1]
                r2 = random.random()  # r2 is a random number in [0,1]

                A1 = 2 * a * r1 - a
                # Equation (3.3)
                C1 = 2 * r2
                # Equation (3.4)

                D_alpha = abs(C1 * Alpha_pos[j] - Positions[i, j])
                # Equation (3.5)-part 1
                X1 = Alpha_pos[j] - A1 * D_alpha
                # Equation (3.6)-part 1

                r1 = random.random()
                r2 = random.random()

                A2 = 2 * a * r1 - a
                # Equation (3.3)
                C2 = 2 * r2
                # Equation (3.4)

                D_beta = abs(C2 * Beta_pos[j] - Positions[i, j])
                # Equation (3.5)-part 2
                X2 = Beta_pos[j] - A2 * D_beta
                # Equation (3.6)-part 2

                r1 = random.random()
                r2 = random.random()

                A3 = 2 * a * r1 - a
                # Equation (3.3)
                C3 = 2 * r2
                # Equation (3.4)

                D_delta = abs(C3 * Delta_pos[j] - Positions[i, j])
                # Equation (3.5)-part 3
                X3 = Delta_pos[j] - A3 * D_delta
                # Equation (3.5)-part 3

                Positions[i, j] = (X1 + X2 + X3) / 3  # Equation (3.7)
                

        Convergence_curve[l] = Alpha_score

        if l / Max_iter-1 == 1:
            print(
                ["At iteration " + str(l) + " the best fitness is " + str(Alpha_score)]
            )
            print(Alpha_pos)

    timerEnd = time.time()
    s.endTime = time.strftime("%Y-%m-%d-%H-%M-%S")
    s.executionTime = timerEnd - timerStart
    s.convergence = Convergence_curve
    s.optimizer = "GWO"
    s.objfname = objf.__name__
    s.bestIndividual.append(Alpha_pos)


    return s

In [ ]:
A=GWO(MLP, -1, 1, 181, 50, 1000)


GWO is optimizing  "MLP"


In [ ]:
print(A.convergence[999])

0.15067932592857294


In [ ]:
solution=A.bestIndividual[0]
print(solution)

[ 3.81872003e-01 -7.28673623e-03 -7.12196376e-01  8.23473300e-02
  2.63753627e-01  7.87424719e-01  1.03495026e-01 -1.28495253e-02
 -3.67035777e-01 -5.53207469e-01  1.11679122e-02  6.30199045e-02
 -1.43951575e-01 -1.55689003e-01  1.53659351e-01  1.19251376e-02
  9.97387244e-01  5.35532244e-03  4.38793595e-02  8.65895351e-03
 -8.91669415e-02  2.41632902e-02  2.73526103e-01  6.73767324e-02
 -8.54566927e-01 -7.48612799e-01  9.04059080e-02  8.74315448e-01
 -2.11295007e-01  1.89850437e-02 -8.65140582e-01  2.58783423e-01
 -5.55239524e-01 -9.94422535e-01  1.35914247e-01  7.80107665e-02
  9.99639091e-01  2.10706987e-02 -6.50475746e-01  9.99773332e-01
  3.89749641e-02  9.87275884e-01 -1.54360704e-01  2.05868472e-02
 -8.23888542e-04  1.10287634e-01  9.20506575e-04  1.00000000e+00
 -2.74086209e-02 -9.31119125e-04 -5.75354867e-02 -8.30921860e-03
 -3.74932873e-04 -6.30815590e-01 -8.47693304e-01 -1.27274459e-01
  5.95261081e-02 -6.66018404e-01 -3.57316492e-02  7.15116699e-04
 -3.74685332e-01  1.44643

In [ ]:
def Test(HL,OL,W,B,x):
  INL=len(x)
  hidden_layer=np.zeros((HL))
  output_layer=np.zeros((OL))
  witr=0
  bitr=0
  for i in range(HL):
    sum=0
    for j in range(INL):
      sum=sum+(x[j]*W[witr])
      witr=witr+1
    sum=sum.sum()
    hidden_layer[i]=sigmoid(sum+B[bitr])
    bitr=bitr+1

  for i in range(OL):
    output=0
    for j in range(HL):
      output=output+(hidden_layer[j]*W[witr])
      witr = witr + 1
    output=output.sum()
    output_layer[i]=sigmoid(output+B[bitr])
    bitr=bitr+1
  return output_layer




In [ ]:
min_key=min(B.keys(), key=(lambda k: B[k]))

In [ ]:
w=(NIL*HL)+(HL*OL)
b=(NIL*HL)+HL+(HL*OL)+OL    
W=solution[0:w]
B=solution[w:b]
print(len(W))
print(B)

162
[-2.28668223e-01 -2.74445278e-02 -2.61144967e-01  2.58309914e-02
  3.33819422e-02  3.18675482e-02 -6.49913223e-05 -7.62481723e-02
  2.17091314e-01 -1.53839726e-03 -9.81438277e-02  2.01603851e-02
 -6.97449044e-01 -9.49469200e-01 -8.94521833e-04 -1.41371550e-02
 -7.96529327e-03  3.19189481e-02  1.75810100e-01]


In [ ]:
a=Test(HL,OL,W,B,x_test_std[7])
print(a)


[1.]


In [ ]:
R=0
for i in range(len(y_train)):
  p=Test(HL,OL,W,B,x_train_std[i])
  if y_train[i][0]==0:
    if p[0]<0.5:
      R+=1
  if y_train[i][0]==1:
    if p[0]>0.5:
      R+=1


In [ ]:
print((R*100)/len(y_train))

78.29861111111111


In [ ]:
def prediction(HL,OL,W,B,x):
  predict=[]
  s=[]
  for i in range(len(x)):
    a=Test(HL,OL,W,B,x[i])
    if a[0]>0.5:
      predict.append(1)
    else:
      predict.append(0)
    
    
  
  return predict

In [ ]:
pred=prediction(HL,OL,W,B,x_test_std)
print(f"Predict :{pred}")
y_true = sum(y_test.tolist(), [])
print(f"true y:{y_true}")

acc=0
for i in range(len(y_test)):
  if pred[i]==y_true[i]:
    acc+=1

print(f"accuracy:{acc/len(y_test)*100}%")

Predict :[1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]
true y:[1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1

In [ ]:
p=[[2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [4.760185995141387e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998617, 7.413173583065964e-08], [1.0, 7.49479046619701e-09], [3.295837933196582e-19, 1.0], [4.760185995141387e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [2.735378606784197e-13, 1.0], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [4.7601859951413703e-07, 1.0], [1.0, 5.790537731467655e-12], [0.9999997594005218, 1.7827926890587977e-14], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3777059110728068e-17], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.3441849876730426e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [2.3443797239678989e-07, 1.0], [0.999997046667274, 1.0], [3.4230559529451456e-19, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [2.7353786067445575e-13, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998619, 7.412144614465116e-08], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [3.2958383525020503e-19, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [4.760185995141387e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 5.790537748567615e-12], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [3.295837933196582e-19, 1.0], [4.760185995141387e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [2.735378606784197e-13, 1.0], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [4.7601859963200897e-07, 1.0], [1.0, 7.49479046619701e-09], [2.730081969202577e-13, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998617, 7.413173838927611e-08], [2.824682922217676e-13, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [1.0, 7.494780457447032e-09], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [1.2482862984116327e-18, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.494790466200846e-09], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [4.7601836872318313e-07, 1.0], [2.3443797239678989e-07, 1.0], [0.9999999999999998, 3.6456980727045307e-10], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [2.3443797239678989e-07, 1.0], [3.561895408985392e-19, 1.0], [0.9999999999998617, 7.413141488936635e-08], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [0.999997046667274, 1.0], [1.0, 7.49479046619701e-09], [2.3443797226534723e-07, 1.0], [0.9999999999997193, 1.377389407325726e-17], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [2.735371990190059e-13, 1.0], [1.0, 7.49479046619701e-09], [3.295837933196582e-19, 1.0], [1.0, 5.790537731467655e-12], [0.9999999954925718, 1.1219721194964509e-13], [4.760185995141387e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 5.790537731467655e-12], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [2.735378606754956e-13, 1.0], [2.344379723967903e-07, 1.0], [2.3443797239678989e-07, 1.0], [4.760185995141387e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [4.7601861964494985e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 1.4646590413208634e-12], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [0.9999999999998617, 7.413173838927611e-08], [0.9999999999997193, 1.3773894071084169e-17], [2.3443797239678989e-07, 1.0], [1.0, 7.416891915703537e-09], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [0.9999997593963258, 1.7827783105267858e-14], [2.735378606784197e-13, 1.0], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [4.7601859947178284e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [0.9999999999997216, 1.382760290185998e-17], [1.0, 7.49479046619701e-09], [2.735378606784197e-13, 1.0], [1.0, 7.494787237171255e-09], [0.9999997593963287, 1.7827783200580683e-14], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [2.735378606784197e-13, 1.0], [1.0, 5.790537731467655e-12], [5.186223429698876e-06, 1.0], [1.0, 7.49479046619701e-09], [3.2963276408309863e-19, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [4.760185995141387e-07, 1.0], [0.999999759396326, 1.7827783109247703e-14], [2.3443797239678989e-07, 1.0], [2.3443797239533066e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [3.539949630789169e-14, 1.0], [4.760185995141387e-07, 1.0], [4.760185995141387e-07, 1.0], [0.9999999999997193, 1.3773894071096696e-17], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.735378606784197e-13, 1.0], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 5.399724176787439e-09], [3.2958381859094635e-19, 1.0], [1.0, 5.790537731467655e-12], [2.3443797239678989e-07, 1.0], [2.736071297671768e-13, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998617, 7.413173838927611e-08], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [1.0, 7.494790466196905e-09], [2.3443797239678989e-07, 1.0], [4.760185995141387e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [0.9999997597973544, 1.784154246233934e-14], [4.760185995141387e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 5.790537731467655e-12], [0.9999999999997193, 1.3773916445774456e-17], [1.0, 5.7905377175175346e-12], [1.0, 7.49479046619701e-09], [0.9999997593963258, 1.7827783105640535e-14], [1.0, 7.49479046619701e-09], [0.9999999999997193, 1.3773894071084169e-17], [4.760185994289154e-07, 1.0], [4.760185995141387e-07, 1.0], [1.0, 5.790537731467655e-12], [0.9999999999997193, 1.3773894071084169e-17], [0.9999997594030715, 1.782801427057929e-14], [0.9999999999997193, 1.3773894071084169e-17], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3773894071148567e-17], [2.3443797239678989e-07, 1.0], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998492, 1.8358616739546913e-17], [2.3443797239678989e-07, 1.0], [2.3443792937421956e-07, 1.0], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [2.735378606784197e-13, 1.0], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.494790466200632e-09], [0.9999997593963258, 1.7827783105267858e-14], [0.9999997593963258, 1.7827783105267858e-14], [0.9999999999997193, 1.3773894071084169e-17], [0.9999999999997193, 1.3773894071086518e-17], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [2.8247268646226163e-13, 1.0], [4.760185922590398e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 7.493736721731287e-09], [4.760185995141387e-07, 1.0], [2.3443797239678989e-07, 1.0], [4.760185995142792e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 5.790537731467655e-12], [3.295837933196582e-19, 1.0], [1.0, 5.7905377314674905e-12], [1.0, 5.790539620623675e-12], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.492433652437726e-09], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [4.760185995141387e-07, 1.0], [5.0873878446689e-18, 1.0]]
for i in range(len(pred)):
  if p[i][0] and p[i][1] >0.5:
    print(i)
  